In [17]:
import json
from IPython.display import clear_output, display, Markdown

# Define parameters.

PROMPTS = "../data/prompts/apibeautiful/taskvine_apibeautiful_gpt4o.json"

In [18]:
# Analyze a singular prompt.

def analyze(prompt: dict):
    clear_output(wait=True)
    print(prompt["content"])
    display(Markdown(prompt["response"]))
    return input("Analysis: ")

In [ ]:
### Load prompts from file.
with open(PROMPTS, "r") as fp:
    prompts = json.load(fp)

# Perform the zeroshot prompting.
for prompt in prompts["prompts"]:
    if True or "analysis" not in prompt or input("Override analysis? (y/n)").lower() in ["y", "yes"]:
        analysis = analyze(prompt)
        prompt["analysis"] = analysis

Write a Python TaskVine application to perform matrix inversion and transposition. The matrix computations must be defined in Python functions and added to a TaskVine library called "matrix_library". The user must specify the size, m x n,  of the matrix and the number of matrices, i, on which to perform computation. The application must randomly generate i matrices and for each matrix, print the matrix, print the inverse matrix, and print the transposed matrix.


```python
import numpy as np
from ndcctools.taskvine.manager import Manager
from ndcctools.taskvine.task import FunctionCall

# Define matrix operations
def invert_matrix(matrix):
    return np.linalg.inv(matrix)

def transpose_matrix(matrix):
    return np.transpose(matrix)

# Main function
def main(matrix_size, num_matrices):
    # Create a TaskVine manager
    manager = Manager(port=0, name="matrix_operations", shutdown=True)

    # Create a library from the matrix functions
    manager.create_library_from_functions(
        library_name="matrix_library",
        function_list=[invert_matrix, transpose_matrix],
        poncho_env=None,
        init_command=None,
        add_env=False,
        hoisting_modules=[],
        exec_mode='direct',
        library_context_info=None
    )

    # Generate random matrices and perform operations
    for _ in range(num_matrices):
        matrix = np.random.rand(matrix_size[0], matrix_size[1])
        print("Original Matrix:\n", matrix)

        # Create tasks for inversion and transposition
        invert_task = FunctionCall("matrix_library", "invert_matrix", args=(matrix,), kwargs={})
        transpose_task = FunctionCall("matrix_library", "transpose_matrix", args=(matrix,), kwargs={})

        # Submit tasks to the manager
        manager.submit(invert_task)
        manager.submit(transpose_task)

        # Wait for tasks to complete and get results
        invert_result = manager.wait_for_task_id(invert_task.id(), timeout="wait_forever").output()
        transpose_result = manager.wait_for_task_id(transpose_task.id(), timeout="wait_forever").output()

        # Print results
        print("Inverted Matrix:\n", invert_result)
        print("Transposed Matrix:\n", transpose_result)

    # Shutdown the manager
    manager.shutdown()

# Example usage
if __name__ == "__main__":
    matrix_size = (3, 3)  # Example size
    num_matrices = 2      # Example number of matrices
    main(matrix_size, num_matrices)
```

This code defines two functions for matrix inversion and transposition, creates a TaskVine manager, and uses it to perform these operations on randomly generated matrices. The results are printed for each matrix. Adjust `matrix_size` and `num_matrices` as needed.